## How do Associations and Attributes Work?

In [1]:
from itertools import product
from operator import itemgetter

import yaml
from z3 import (And, Const, Consts, DatatypeSortRef, Exists, ForAll,
                FuncDeclRef, Implies, Model, Not, Or, Solver, sat, unsat)

from mc_openapi.doml_mc import DOMLVersion
from mc_openapi.doml_mc.intermediate_model.metamodel import (
    parse_inverse_associations, parse_metamodel)
from mc_openapi.doml_mc.xmi_parser.doml_model import (parse_doml_model,
                                                      parse_xmi_model)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations, assert_im_attributes,
    def_elem_class_f_and_assert_classes, mk_attr_data_sort, mk_elem_sort_dict,
    mk_stringsym_sort_dict)
from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel, def_attribute_rel, mk_association_sort_dict,
    mk_attribute_sort_dict, mk_class_sort_dict)
from mc_openapi.doml_mc.z3encoding.types import Refs

In [2]:
with open("../assets/doml_meta_v2.0.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)

**You can change here the input DOML file**

In [3]:
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0.domlx"
doml_document_path = "../../tests/doml/v2.0/nginx-openstack_v2.0_wrong_vm_iface.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2.0_wrong_iface_uniq.domlx"
# doml_document_path = "../../tests/doml/saas_no_https_rule.domlx"
# doml_document_path = "../../tests/doml/saas_https_no_attrs.domlx"
# doml_document_path = "../../tests/doml/openstack_template.domlx"

In [4]:
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im, _ = parse_doml_model(doc, DOMLVersion.V2_0)

We need to initialize each time the Solver context before iterating,
since an unbound variable is an element, and elements are an EnumSort,
and EnumSorts cannot be modified and depend on the solver context.

The following code is stuff that is already present in the `IntermediateModelChecker`.

In [5]:
from typing import Dict

Context = Dict

In [6]:
from mc_openapi.doml_mc.intermediate_model.metamodel import get_mangled_attribute_defaults


def initialize_solver(
    unbound_elems_quantity: int = 0,
    unbound_values_quantity: int = 0,
    requirements: list = []
) -> Context:
    ctx = dict()
    
    ctx["solver"] = Solver()

    ctx["class_sort"], ctx["class_"] = mk_class_sort_dict(mm, ctx["solver"].ctx)
    ctx["assoc_sort"], ctx["assoc"] = mk_association_sort_dict(mm, ctx["solver"].ctx)
    ctx["attr_sort"], ctx["attr"] = mk_attribute_sort_dict(mm, ctx["solver"].ctx)
    ctx["str_sort"], ctx["str"] = mk_stringsym_sort_dict(im, mm, ctx["solver"].ctx)
    ctx["attr_data_sort"] = mk_attr_data_sort(ctx["str_sort"], ctx["solver"].ctx)

    ctx["unbound_elems"] = [f"unbound_elem_{i}" for i in range(unbound_elems_quantity)]

    # Takes a list of strings and creates an Enum out of 'em
    ctx["elem_sort"], ctx["elem"] = mk_elem_sort_dict(im, ctx["solver"].ctx, ctx["unbound_elems"])

    ub_val_names = [f"unbound_val_{i}" for i in range(unbound_values_quantity)]
    ctx["unbound_values"] = {
        name : ctx["attr_data_sort"].placeholder for name in ub_val_names
    }
    # Examples of values that can go in unbound_values:
    # ctx["attr_data_sort"].int(42), # ok
    # ctx["attr_data_sort"].bool(True), # ok
    # ctx["attr_data_sort"].str("x"), # cant do: it accept a ctx["str"][<str_key>] as input
    # Const("x", ctx["attr_data_sort"]) # cant do: it is a symbolic value that cannot be converted to a BoolRef expression

    ctx["elem_class_f"] = def_elem_class_f_and_assert_classes(
        im,
        ctx["solver"],
        ctx["elem_sort"],
        ctx["elem"],
        ctx["class_sort"],
        ctx["class_"]
    )
    
    # attr_rel :: (elem_sort, attr_sort, attr_data_sort) -> BoolRef
    ctx["attr_rel"] = def_attribute_rel(
        ctx["attr_sort"],
        ctx["elem_sort"],
        ctx["attr_data_sort"]
    )

    assert_im_attributes(
        ctx["attr_rel"],
        ctx["solver"],
        im,
        mm,
        ctx["elem"],
        ctx["attr_sort"],
        ctx["attr"],
        ctx["attr_data_sort"],
        ctx["str"]
    )

    # assoc_rel :: (elem_sort, assoc_sort, elem_sort) -> BoolRef
    ctx["assoc_rel"] = def_association_rel(
        ctx["assoc_sort"],
        ctx["elem_sort"]
    )
    
    assert_im_associations(
        ctx["assoc_rel"],
        ctx["solver"],
        {k: v for k, v in im.items() if k not in ctx["unbound_elems"]},
        ctx["elem"],
        ctx["assoc_sort"],
        ctx["assoc"],
    )

    # Add requirements
    for req in requirements:
        req(ctx)

    return ctx

In [7]:
initialize_solver()
print()

== None ==
ForAll([a, d],
       attribute(elem_140187603074384, a, d) ==
       Or(And(a == commons_Property::key,
              d == str(str_10_source_code)),
          And(a == commons_SProperty::value,
              d == str(str_22__usr_share_nginx))))
== nginx ==
ForAll([a, d],
       attribute(elem_140187603074256, a, d) ==
       Or(And(a == commons_DOMLElement::name,
              d == str(str_28_nginx)),
          And(a == commons_DOMLElement::description,
              d == placeholder),
          And(a ==
              application_SoftwareComponent::isPersistent,
              d == bool(False)),
          And(a ==
              application_SoftwareComponent::licenseCost,
              d == placeholder),
          And(a == application_SoftwareComponent::configFile,
              d == placeholder)))
== app ==
ForAll([a, d],
       attribute(elem_140187603072976, a, d) ==
       Or(And(a == commons_DOMLElement::name,
              d == str(str_30_app)),
          And(a == commo